In [326]:
dataset = 'A2_EDA_Residential.csv'

In [327]:
import polars as pl

In [328]:
def new_df():
    df = pl.read_csv(dataset)
    df = (
        df.with_columns(
            pl.col("date").str.split("-").list.get(0).cast(int).alias("date")
        )
    )
    df = df.with_columns(
    (pl.col("flip_horizon") / 365.0).alias("flip_horizon_years")
    )
    df = df.with_columns(
        (pl.col("price_diff") / pl.col("flip_horizon_years")).alias("yearly_return")
    )
    return df

df = new_df()
df
# df["year"].max()
# df = df.filter(year = 2022)

address,street,proptype,date,price,prsldate,prslpr,mortgage,usecode,usage,style,yearbuilt,totrooms,bathrooms,bedrooms,lotsize,intersf,lasqbin,amenity,zip,lat,lon,year,cash_sale,buyer_llc_ind,buyer_bus_ind,buyer_trst_ind,buyer_bnk_ind,buyer_gse_ind,buyer_gov_ind,seller_llc_ind,seller_llp_ind,seller_bus_ind,seller_trst_ind,seller_bnk_ind,seller_gse_ind,seller_gov_ind,flip_horizon,flip_ind,buy_side_flip,sell_side_flip,price_diff,price_diff_pch,month_horizon,flip_term,buy_horizon,investor_type_purchase_count,investor_type_sale_count,investor_type_purchase_llc,investor_type_sale_llc,investor_type_purchase_building,investor_type_sale_building,avg_annual_value,investor_type_purchase_value,investor_type_sale_value,current_owner,restype_R1F,tot_owned,buyer_purchases,R1F_total,investor_type_purchase,investor_type_sale,inv_to_inv,flip_horizon_years,yearly_return
str,str,str,i64,f64,str,f64,f64,i64,str,str,i64,i64,i64,i64,i64,i64,str,str,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,str,str,str,str,str,str,str,str,str,f64,str,str,i64,i64,i64,i64,i64,str,str,i64,f64,f64
"""48 Commercial Wharf #48""","""Commercial Wharf""","""RCD""",2012,899000.0,"""07/08/11""",830000.0,0.0,102,"""CONDOMINIUM""","""MID-RISE""",1875,5,2,2,1043,1043,"""1001:1100""","""U""",2110,42.362099,-71.050169,2012,1,0,0,0,0,0,0,0,0,0,0,0,0,0,431,1,0,1,69000.0,8.31,"""14""","""Long-term""","""2017-09-11""","""Non-count investor""","""Non-count investor""","""Non-Small LLC""","""Non-Small LLC""","""Non-building investor""","""Non-building investor""",39086.956522,"""Non-value investor""","""Non-value investor""",1,0,1,1,0,"""Non-investor""","""Non-investor""",0,1.180822,58433.87471
"""6 Arlington St #9""","""Arlington St""","""RCD""",2008,4.867787e6,null,0.0,2.25e6,102,"""CONDOMINIUM""","""MID-RISE""",1910,7,4,3,3319,3319,"""3301:3400""","""U""",2116,42.354791,-71.072323,2008,0,0,0,1,0,0,0,1,0,0,0,0,0,0,null,0,0,0,null,null,null,"""Non-flip""","""2013-10-23""","""Non-count investor""","""Non-count investor""","""Non-Small LLC""","""Small LLC""","""Non-building investor""","""Non-building investor""",211642.913043,"""Small""","""Small""",0,0,0,1,0,"""Small""","""Small""",1,null,null
"""1 Charles St S #902""","""Charles St S""","""RCD""",2014,807500.0,"""11/06/08""",665000.0,160000.0,102,"""CONDOMINIUM""","""HIGH-RISE""",2004,3,2,1,926,926,"""901:1000""","""U""",2116,42.351787,-71.067367,2014,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2204,0,0,0,142500.0,21.43,"""24+""","""Non-flip""","""2019-11-19""","""Non-count investor""","""Non-count investor""","""Small LLC""","""Non-Small LLC""","""Non-building investor""","""Non-building investor""",35108.695652,"""Non-value investor""","""Non-value investor""",0,0,0,1,0,"""Small""","""Non-investor""",0,6.038356,23599.137931
"""1 Avery St #31B""","""Avery St""","""RCD""",2013,3.475e6,"""08/29/08""",3.05e6,0.0,102,"""CONDOMINIUM""","""HIGH-RISE""",2000,7,5,3,2667,2667,"""2601:2700""","""U""",2111,42.35337,-71.062515,2013,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1826,0,0,0,425000.0,13.93,"""24+""","""Non-flip""","""2018-08-29""","""Non-count investor""","""Non-count investor""","""Non-Small LLC""","""Non-Small LLC""","""Non-building investor""","""Non-building investor""",151086.956522,"""Small""","""Small""",1,0,1,1,0,"""Small""","""Small""",1,5.00274,84953.450164
"""1 Avery St #10D""","""Avery St""","""RCD""",2009,845000.0,"""03/29/07""",845000.0,0.0,102,"""CONDOMINIUM""","""HIGH-RISE""",2000,4,2,2,1087,1087,"""1001:1100""","""U""",2111,42.35337,-71.062515,2009,1,0,0,1,0,0,0,0,0,1,0,0,0,0,null,0,0,0,0.0,0.0,null,"""Non-flip""","""2014-04-30""","""Non-count investor""","""Non-count investor""","""Non-Small LLC""","""Non-Small LLC""","""Non-building investor""","""Non-building investor""",36739.130435,"""Non-value investor""","""Non-value investor""",0,0,0,1,0,"""Non-investor""","""Non-investor""",0,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…

In [329]:
# df = df.filter(pl.col("flip_ind") == 0)
# df.head()

In [330]:
# All transactions, all players, what is the flip rate
non_investor_sales = df.filter(pl.col("investor_type_sale") == "Non-investor")
non_investor_sales = non_investor_sales.with_columns(
    pl.lit(False).alias("isLarge")
)

non_investor_sales_sell_side_flip = non_investor_sales.filter(pl.col("sell_side_flip") == True)
non_investor_flip_rate = non_investor_sales_sell_side_flip.shape[0] / non_investor_sales.shape[0]
print(len(non_investor_sales))
print(len(non_investor_sales_sell_side_flip))
print(non_investor_flip_rate * 100)


33070
883
2.6700937405503478


In [ ]:
# All transactions, all players, what is the flip rate
target_set = ["Large", "Institutional"]
rich_investor_sales =  df.filter(pl.col("investor_type_sale").is_in(target_set))
rich_investor_sales = rich_investor_sales.with_columns(
    pl.lit(True).alias("isLarge")
)

rich_investor_sales_sell_side_flips = rich_investor_sales.filter(pl.col("sell_side_flip") == True)
rich_investor_flip_rate = rich_investor_sales_sell_side_flips.shape[0] / rich_investor_sales.shape[0]
print(len(rich_investor_sales))
print(len(rich_investor_sales_sell_side_flips))
print(rich_investor_flip_rate * 100)


all = pl.concat([non_investor_sales, rich_investor_sales])
all = all.with_columns(
    (pl.col("prslpr")).alias("original_price")
)

all_flips = pl.concat([non_investor_sales_sell_side_flip, rich_investor_sales_sell_side_flips])
all_flips = all_flips.group_by("isLarge").agg(
    (pl.col("yearly_return")).median().alias("median_yearly_return"),
    (pl.col("yearly_return") / pl.col("prslpr")).median().alias("median_yearly_return_rate"),
)
all_flips


3302
246
7.450030284675954


isLarge,median_yearly_return,median_yearly_return_rate
bool,f64,f64
false,51798.679868,0.077385
true,188712.658442,0.204116
